# Twin Identifier Analysis

This notebook looks for local unique identifiers that are the same in different biomedical ontologies, databases, and identifier spaces. The fact that there are lots of overlaps shows that the need for well-defined prefixes is incredibly important.

In [1]:
import json
from itertools import combinations
from operator import itemgetter

import bioregistry
import networkx as nx
import pandas as pd
import pyobo
from tqdm.auto import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm

Extract the list of local unique identifiers from all resources gettable from PyOBO. Note that these local unique identifiers are normalized by the Bioregistry to remove potential redundant prefixes.

In [3]:
skip = {"umls", "kegg", "icd10", "icd11"}
resources = [
    resource
    for resource in bioregistry.resources()
    if (resource.has_download() or pyobo.has_nomenclature_plugin(resource.prefix)) 
    and not resource.is_deprecated()
    and resource.prefix not in skip
]
assert "mesh" in {r.prefix for r in resources}

In [4]:
ids = {}
with logging_redirect_tqdm():
    it = tqdm(resources, unit="prefix", unit_scale=True)
    failed_prefixes = set()
    succeeded_prefixes = set()
    for resource in it:
        it.set_postfix(failed=len(failed_prefixes), succeeded=len(succeeded_prefixes), current=resource.prefix)
        try:
            resource_ids = pyobo.get_ids(resource.prefix, strict=False)
        except Exception:
            tqdm.write(f"failed on {resource.prefix}")
            failed_prefixes.add(resource.prefix)
            continue
        else:
            if resource_ids:
                ids[resource.prefix] = resource_ids
            succeeded_prefixes.add(resource.prefix)

  0%|          | 0.00/362 [00:00<?, ?prefix/s]

failed on ado


INFO: [2023-02-10 13:39:21] pystow.utils - downloading with urllib from http://purl.allotrope.org/voc/afo/merged-OLS/REC/2019/05/10 to /Users/cthoyt/.data/pyobo/raw/afo/2019-05-10/10


failed on aeon
failed on afo
failed on apaonto
failed on apollosv


[Fatal Error] :67773:173: The reference to entity "idAhol" must end with the ';' delimiter.


...

failed on uniprot.keyword


INFO: [2023-02-10 14:00:09] pystow.utils - downloading with urllib from https://github.com/obophenotype/upheno/blob/master/upheno.obo to /Users/cthoyt/.data/pyobo/raw/upheno/2017-10-31/upheno.obo


failed on uniprot.location
failed on upheno
failed on vido
failed on vo
failed on vso


INFO: [2023-02-10 14:00:31] pyobo.getters - [zfin] caching nomenclature plugin
INFO: [2023-02-10 14:00:31] pystow.utils - downloading with urllib from https://zfin.org/downloads/zdb_history.txt to /Users/cthoyt/.data/pyobo/raw/zfin/9 Feb 2023/alts.tsv
INFO: [2023-02-10 14:00:33] pystow.utils - downloading with urllib from https://zfin.org/downloads/human_orthos.txt to /Users/cthoyt/.data/pyobo/raw/zfin/9 Feb 2023/human_orthos.txt
INFO: [2023-02-10 14:00:36] pystow.utils - downloading with urllib from https://zfin.org/downloads/mouse_orthos.txt to /Users/cthoyt/.data/pyobo/raw/zfin/9 Feb 2023/mouse_orthos.txt
INFO: [2023-02-10 14:00:39] pystow.utils - downloading with urllib from https://zfin.org/downloads/fly_orthos.txt to /Users/cthoyt/.data/pyobo/raw/zfin/9 Feb 2023/fly_orthos.txt
INFO: [2023-02-10 14:00:40] pystow.utils - downloading with urllib from https://zfin.org/downloads/gene.txt to /Users/cthoyt/.data/pyobo/raw/zfin/9 Feb 2023/gene.txt
INFO: [2023-02-10 14:00:42] pystow.utils

failed on zfin


In [5]:
rows = []
prefixes_encountered = set()
total = len(ids) * (len(ids) - 1) / 2
min_terms = 10
for (a_prefix, a), (b_prefix, b) in tqdm(
    combinations(ids.items(), 2), total=total, unit_scale=True
):
    len_a, len_b = len(a), len(b)
    if len_a < min_terms or len_b < min_terms:
        continue

    prefixes_encountered.add(a_prefix)
    prefixes_encountered.add(b_prefix)

    intersection = set(a).intersection(b)
    if not intersection:
        continue

    n_intersection = len(intersection)
    overlap = n_intersection / min(len_a, len_b)
    rows.append((a_prefix, b_prefix, len_a, len_b, n_intersection, round(overlap, 5)))

# Sort descending by normalized overlap
rows = sorted(rows, key=itemgetter(5), reverse=True)
overlap_df = pd.DataFrame(
    rows,
    columns=[
        "source_prefix",
        "target_prefix",
        "source_size",
        "target_size",
        "absolute_overlap",
        "normalized_overlap",
    ],
)
overlap_df.to_csv("edges.tsv", sep="\t", index=False)
overlap_df

  0%|          | 0.00/14.0k [00:00<?, ?it/s]

,source_prefix,target_prefix,source_size,target_size,absolute_overlap,normalized_overlap
0,apo,lbo,309,1108,309,1.00000
1,apo,sbo,309,694,309,1.00000
2,bfo,bto,35,6511,35,1.00000
3,bfo,chmo,35,2908,35,1.00000
4,bfo,cmo,35,3025,35,1.00000
...,...,...,...,...,...,...
5325,doid,efo,11207,15540,1,0.00009
5326,doid,mp,11207,13569,1,0.00009
5327,fbbt,tto,17570,38640,1,0.00006
5328,fbbt,xpo,17570,20061,1,0.00006


In [6]:
prefix_rows = [
    (
        prefix,
        bioregistry.get_name(prefix),
        bioregistry.get_example(prefix),
        len(ids[prefix]),
    )
    for prefix in sorted(prefixes_encountered)
]
prefix_df = pd.DataFrame(
    prefix_rows, columns=["prefix", "name", "example_luid", "n_luids"]
)
prefix_df.to_csv("nodes.tsv", sep="\t", index=False)
prefix_df

,prefix,name,example_luid,n_luids
0,agro,Agronomy Ontology,00020007,772
1,aism,Ontology for the Anatomy of the Insect Skeleto...,0000027,553
2,amphx,The Amphioxus Development and Anatomy Ontology,1000160,399
3,antibodyregistry,Antibody Registry,493771,1898783
4,apo,Ascomycete phenotype ontology,0000184,309
...,...,...,...,...
153,xpo,Xenopus Phenotype Ontology,0100002,20061
154,zeco,Zebrafish Experimental Conditions Ontology,0000171,161
155,zfa,Zebrafish anatomy and development ontology,0005926,3079
156,zfs,Zebrafish developmental stages ontology,0000050,54


In [7]:
graph = nx.Graph()
for prefix, name, _example, size in prefix_rows:
    graph.add_node(
        prefix,
        name=name,
        size=size,
        prefix=prefix,
        description=bioregistry.get_description(prefix),
    )
for source, target, _, _, absolute_overlap, normalized_overlap in rows:
    graph.add_edge(
        source,
        target,
        weight=normalized_overlap,
        absolute_overlap=absolute_overlap,
    )

In [8]:
nx.write_graphml_lxml(graph, "graph.graphml")

In [9]:
with open("graph.cytoscape.json", "w") as file:
    json.dump(nx.cytoscape_data(graph), file, indent=2)